<a href="https://colab.research.google.com/github/kridtapon/Chop-TrendFlow-STC-/blob/main/Chop_TrendFlow_STC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Schaff Trend Cycle (STC)
def calculate_stc(df, short_ema=12, long_ema=26, stoch_period=10, signal_period=3):
    """
    Calculate Schaff Trend Cycle (STC).
    """
    df['Short_EMA'] = df['Close'].ewm(span=short_ema).mean()
    df['Long_EMA'] = df['Close'].ewm(span=long_ema).mean()

    macd = df['Short_EMA'] - df['Long_EMA']
    df['MACD_Signal'] = macd.ewm(span=9).mean()

    macd_diff = macd - df['MACD_Signal']
    lowest_macd = macd_diff.rolling(stoch_period).min()
    highest_macd = macd_diff.rolling(stoch_period).max()

    df['STC'] = 100 * ((macd_diff - lowest_macd) / (highest_macd - lowest_macd))
    df['STC_Signal'] = df['STC'].rolling(signal_period).mean()

    return df['STC']

# Function to calculate Choppiness Index (CHOP)
def calculate_choppiness_index(df, period=14):
    """
    Calculate Choppiness Index (CHOP).
    """
    high_low_range = df['High'] - df['Low']
    atr = high_low_range.rolling(window=period).sum()

    max_high = df['High'].rolling(window=period).max()
    min_low = df['Low'].rolling(window=period).min()

    chop = 100 * np.log10(atr / (max_high - min_low)) / np.log10(period)

    return chop

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download historical data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate STC and CHOP
df['STC'] = calculate_stc(df)
df['CHOP'] = calculate_choppiness_index(df)

# Define Entry and Exit signals
df['Entry'] = (df['STC'] > 25) & (df['STC'].shift(1) <= 25) & (df['CHOP'] < 40)
df['Exit'] = (df['STC'] < df['STC'].shift(1)) | (df['CHOP'] > 60)

# Filter data for backtesting (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                             87202.07259
Total Return [%]                       -12.797927
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                       5260.229397
Max Drawdown [%]                        32.078695
Max Drawdown Duration                      1231.0
Total Trades                                   31
Total Closed Trades                            31
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            41.935484
Best Trade [%]                          19.575137
Worst Trade [%]                        -14.438734
Avg Winning Trade [%]                    5.128569
Avg Losing Trade [%]                    -4.098468
